# Asynchonous Operation
DNAC is a collection of stateless microservices.   All of the PUT/POST/DELETE operations are non-blocking (asynchronous).


When an API resquest, the user gets a task that they need to poll until it completes.  The task will contain status information about the result.


### Login
This is a simplified login approach, where a module is used.

In [ ]:
from login import login
import json
dnac = login()

## Pathtrace
The pathtrace service traces the path between two hosts across the network.  The first call to start a pathtrace will return a task.


In [ ]:
flowTask = dnac.flowanalysis.initiateFlowAnalysis(flowAnalysisRequest={"sourceIP": '10.10.22.114',
                                                                           "destIP" : '10.10.22.98',
                                                                           "periodicRefresh" : False})

The response contains a 'taskId' to monitor the status of the request

In [ ]:
print (json.dumps(dnac.serialize(flowTask), indent=4))

### Tasks
The 'wait_for_task_complete' method takes a task and an optional timout and polling_frequency.  It monitors the status of the task, and returns when the task is complete (or the timeout is reached).

In [ ]:
# extrace the taskId
taskId = flowTask.response.taskId

# wait_for_task_complete is a utility function that polls APIC-EM until the task is completed.  
# You can provide an optional timeout and polling interval
taskStatus = dnac.task_util.wait_for_task_complete(flowTask, timeout=15, poll_frequency=5)

print(json.dumps(dnac.serialize(taskStatus), indent=4))

### 'progress' attribute
The 'progress' attribute from the completed task is often used to provide information about the request.  In the case of a POST, it is usually an 'id' of the nely created resource.

In [ ]:
# this is the id of the pathtrace result that we need to GET.
flowId = taskStatus.progress

# this will get the pathTrace
pathTrace = dnac.flowanalysis.getFullFlowAnalysisResult(flowAnalysisId=flowId)

print (json.dumps(dnac.serialize(pathTrace.response),indent=4))

This result can be pretty printed

In [ ]:
for device in pathTrace.response.networkElementsInfo:
    print("{name}: {ip}\n".format(name=device.name, ip=device.ip))

<hr>
<font color=blue>
<h2> Challenge</h2>
Change the call to request device statistics and display those.
<p>
<i><b> Hint: "inclusions" : ["DEVICE-STATS"] needs to be added to the payload body</b></i>
<p>
<b> Bonus, also display the version of code for the device
</font>
<hr>